In [ ]:
## tried to estimate the lstm of paper, but the accuracy is lower than expected
import json
import pandas as pd
import re
import nltk
import numpy as np
#nltk.download('all')
from nltk.tokenize import sent_tokenize # tokenizes sentences
from nltk.stem import PorterStemmer
from nltk.tag import pos_tag
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.util import ngrams
import matplotlib.pyplot as plt

In [ ]:
wordsList = np.load('wordsList.npy')
print('Loaded the word list!')
wordsList = wordsList.tolist() #Originally loaded as numpy array
wordsList = [word.decode('UTF-8') for word in wordsList] #Encode words as UTF-8
wordVectors = np.load('wordVectors.npy')
print ('Loaded the word vectors!')

In [ ]:
with open('pmid-pdf.json','r',encoding='UTF-8') as f:data_full = json.load(f)

In [ ]:
df=pd.DataFrame(data_full)

In [ ]:
train=pd.read_csv('training.csv', encoding = "ISO-8859-1")

In [ ]:
features = ['randomized', 'trial', 'randomization', 'controlled', 'placebo', 'trials', 'group', 'baseline', 'intervention', 'double', 'study', 'dose', 'design', 'studies', 'treatment', 'search', 'effect', 'week', 'published', 'participants', 'primary', 'center', 'assigned', 'received', 'outcome', 'included', 'inclusion', 'clinical', 'end', 'using', 'groups', 'prior', 'eligible', 'used', 'one', 'analysis', 'patients', 'visit', 'informed', 'day', 'mg', 'administered', 'days', 'randomly', 'criteria', 'weeks']

In [ ]:
columns = ['pmid','flag','method']
new_df = pd.DataFrame(columns=columns)
randome = ['randomiz']
nonrandome = ['non-random','nonrandom']
randome_count = 0
nonrandome_count = 0
randome_count_1 = 0
nonrandome_count_1 = 0
randome_count_2 = 0
nonrandome_count_2 = 0
flag = 0
k=0
p1 = 1
p2 = 1000
p= 0
use_key_words = 0 # if the feature key words are used to select sentences 
for i in range(len(df)):
    if (i % 500 == 0):
        print("processed ..", i)
    pmid = df['pmid'][i]
    body = df['body'][i]
    method = ''
    method_comp = ''
    full_body = ''
    randome_flag = 0
    nonrandome_flag = 0
    randome_flag_1 = 0
    flag = 0
    for j in range(len(body)):
        temp = body[j]
        full_body = full_body + "\n" + temp['content']
        if (re.search('method',temp['header'],re.I)):
            method = method + temp['content'] + " "
    if not method:
        continue
    # fetch only sentences in method with top feature words
    if use_key_words == 1:
        sentence = []
        sentence = method.split(".")
        for m in range(len(sentence)):
            for n in range(len(features)):
                if re.search(features[n],sentence[m],re.I):
                    method_comp = method_comp + sentence[m] + "."
                    break
        if not method_comp:
            continue
        method = method_comp
    abstract = str(df['_abstract'][i])
    title = df['title'][i]
    for j in range(len(randome)):
        my_re1 = r"^" + re.escape(randome[j])
        my_re2 = r" " + re.escape(randome[j])
        if (re.search(my_re1,method,re.I) or re.search(my_re2,method,re.I)):
            randome_count = randome_count + 1
            randome_flag = 1
            randome_flag_1 = 1
        if (re.search(my_re1,abstract,re.I) or re.search(my_re2,abstract,re.I)):
            randome_flag_1 = 1
        if (re.search(my_re1,full_body,re.I) or re.search(my_re2,full_body,re.I)):
            randome_flag_1 = 1        
    for j in range(len(nonrandome)):
        my_re = nonrandome[j]
        if (re.search(my_re,method,re.I)):
            nonrandome_count = nonrandome_count + 1
            nonrandome_flag = 1
        #if (re.search(my_re,abstract,re.I)):
        #    nonrandome_count_1 = nonrandome_count_1 + 1
        #    nonrandome_flag = 1
        #if (re.search(my_re,full_body,re.I)):
        #    nonrandome_count_2 = nonrandome_count_2 + 1
        #    nonrandome_flag = 1
    if (randome_flag == 1 and nonrandome_flag != 1):
        flag = 1
        new_df.loc[k] = [pmid,flag,method]
        k=k+1
    elif (nonrandome_flag == 1 or randome_flag_1 != 1):
        flag = 0
        p=p+1
        if p >= p1 and p <= p2:
            new_df.loc[k] = [pmid,flag,method]
            k=k+1
print(new_df)

In [ ]:
print(new_df['method'][1847])

In [ ]:
numWords = []
for i in range(len(new_df)):
    numWords.append(len(new_df['method'][i].split()))
%matplotlib inline
plt.hist(numWords, 50)
plt.xlabel('Sequence Length')
plt.ylabel('Frequency')
plt.axis([0, 1000, 0, 300])
plt.show()

In [ ]:
maxSeqLength = 500
numFiles = len(new_df)

In [ ]:
eng_stopwords = set(stopwords.words("english"))
def review_cleaner(review,lemmatize=False,stem=False):
    '''
    Clean and preprocess a review.

    1. Use regex to remove all special characters (only keep letters)
    2. Make strings to lower case and tokenize / word split reviews
    3. Remove English stopwords
    4. Rejoin to one string
    '''
    ps = PorterStemmer()
    wnl = WordNetLemmatizer()
    cleaned_review=''
    #1. Remove punctuation
    review = re.sub("[^a-zA-Z0-9]", " ",review)
    
    #2. Tokenize into words (all lower case)
    review = review.lower().split()

    #3. Remove stopwords
            
    clean_review=[]
    for word in review:
        if word not in eng_stopwords:
            if lemmatize is True:
                word=wnl.lemmatize(word)
            elif stem is True:
                if word == 'oed':
                    continue
                word=ps.stem(word)
            clean_review.append(word)

    #6. Join the review to one sentence    
    review_processed = ' '.join(clean_review)
    return(review_processed)

In [ ]:
ids = np.zeros((numFiles, maxSeqLength), dtype='int32')
fileCounter = 0
for i in range(len(new_df)):
    indexCounter = 0
    line=new_df['method'][i]
    cleanedLine = review_cleaner(line,lemmatize=False,stem=False)
    split = cleanedLine.split()
    for word in split:
        try:
            ids[fileCounter][indexCounter] = wordsList.index(word)
        except ValueError:
            ids[fileCounter][indexCounter] = 399999 #Vector for unkown words
        indexCounter = indexCounter + 1
        if indexCounter >= maxSeqLength:
            break
    fileCounter = fileCounter + 1
    if (fileCounter % 100 == 0):
        print("loaded ..", fileCounter, "files")
np.save('idsMatrix', ids)

In [ ]:
ids = np.load('idsMatrix.npy')

In [ ]:
# QC testing an example
print(wordsList.index('randomized'))
print(ids[1800][0:20])
print(new_df['flag'][1800])
method = review_cleaner(new_df['method'][1800],lemmatize=False,stem=False)
print(method)
split = method.split()
for word in split:
    try:
        print(wordsList.index(word))
    except ValueError:
        print('399999')

In [ ]:
batchSize = 72 # changing to 96 or 48, performance is worse
lstmUnits = 64 # number of hidden layers, 96 similar/overfitting/flutuat, 48 is worse
numClasses = 2 # convert the classic 1/0 boolean to two columnes (1,0) (0,1)
iterations = 1000
learning_rate = 0.001 # default is 0.001, if loss is unstable, decrease. if decrease is too slow, increase

In [ ]:
from sklearn.model_selection import train_test_split
size=len(new_df)
new_df['reverse'] = 1- new_df['flag'].values
y=new_df[['flag','reverse']].values.reshape(size,2) #astype('int')
X_train, X_test, y_train, y_test = train_test_split(ids, y, random_state=0, test_size=.2)

In [ ]:
print(X_train)

In [ ]:
import tensorflow as tf
tf.reset_default_graph()

labels = tf.placeholder(tf.float32, [batchSize, numClasses])
input_data = tf.placeholder(tf.int32, [batchSize, maxSeqLength])

In [ ]:
numDimensions = 50 # 50-300, change it to 100 does not improve performance
data = tf.nn.embedding_lookup(wordVectors,input_data)
data = tf.Variable(tf.zeros([batchSize, maxSeqLength, numDimensions]),dtype=tf.float32)
data = tf.nn.embedding_lookup(wordVectors,input_data)

In [ ]:
lstmCell = tf.contrib.rnn.BasicLSTMCell(lstmUnits)
lstmCell = tf.contrib.rnn.DropoutWrapper(cell=lstmCell, output_keep_prob=0.95)
value, _ = tf.nn.dynamic_rnn(lstmCell, data, dtype=tf.float32)

In [ ]:
weight = tf.Variable(tf.truncated_normal([lstmUnits, numClasses]))
bias = tf.Variable(tf.constant(0.1, shape=[numClasses]))
value = tf.transpose(value, [1, 0, 2])
last = tf.gather(value, int(value.get_shape()[0]) - 1)
prediction = (tf.matmul(last, weight) + bias)

In [ ]:
correctPred = tf.equal(tf.argmax(prediction,1), tf.argmax(labels,1))
accuracy = tf.reduce_mean(tf.cast(correctPred, tf.float32))

In [ ]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=labels))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

In [ ]:
def getTrainBatch(cycle):
    max_cycle=int(np.floor(len(y_train)/batchSize))
    new_cycle = int(cycle%max_cycle)
    start = new_cycle*batchSize
    end = start + batchSize
    nlabels = y_train[start:end]
    arr = X_train[start:end]
    return arr, nlabels

def getTestBatch(cycle):
    max_cycle=int(np.floor(len(y_test)/batchSize))
    start = int(cycle%max_cycle*batchSize)
    end = start + batchSize
    nlabels = y_test[start:end]
    arr = X_test[start:end]
    return arr, nlabels

In [ ]:
import datetime

sess = tf.InteractiveSession()
tf.summary.scalar('Loss', loss)
tf.summary.scalar('Accuracy', accuracy)
merged = tf.summary.merge_all()
logdir = "tensorboard/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + "/"
writer = tf.summary.FileWriter(logdir, sess.graph)

In [ ]:
sess = tf.InteractiveSession()
saver = tf.train.Saver()
sess.run(tf.global_variables_initializer())

for i in range(iterations):
    #Next Batch of reviews
    nextBatch, nextBatchLabels = getTrainBatch(i)
    sess.run(optimizer, {input_data: nextBatch, labels: nextBatchLabels})
   
    #Write summary to Tensorboard
    if (i % 10 == 0):
        summary = sess.run(merged, {input_data: nextBatch, labels: nextBatchLabels})
        writer.add_summary(summary, i)

    #Save the network every 100 training iterations
    if (i % 100 == 0 and i != 0):
        save_path = saver.save(sess, "models/pretrained_lstm.ckpt", global_step=i)
        print("saved to %s" % save_path)
writer.close()

In [ ]:
sess = tf.InteractiveSession()
saver = tf.train.Saver()
saver.restore(sess, tf.train.latest_checkpoint('models'))

In [ ]:
iterations = 10
for i in range(iterations):
    nextBatch, nextBatchLabels = getTestBatch(i);
    print("Accuracy for this batch:", (sess.run(accuracy, {input_data: nextBatch, labels: nextBatchLabels})) * 100)
    print("Prediction:", (sess.run(tf.argmax(prediction,1), {input_data: nextBatch, labels: nextBatchLabels})))
    print("Labels:", (sess.run(tf.argmax(labels,1), {input_data: nextBatch, labels: nextBatchLabels})))